# Введение

Логистическая регрессия — один из видов линейных классификаторов. Одной из ее особенностей является возможность оценивания вероятностей классов, тогда как большинство линейных классификаторов могут выдавать только номера классов.

Логистическая регрессия использует достаточно сложный функционал качества, который не допускает записи решения в явном виде (в отличие от, например, линейной регрессии). Тем не менее, логистическую регрессию можно настраивать с помощью градиентного спуска.

Мы будем работать с выборкой, содержащей два признака. Будем считать, что ответы лежат в множестве {-1, 1}. Для настройки логистической регрессии мы будем решать следующую задачу:

$\frac{1}{l}\sum^{l}_{i=1}log(1+exp(-y_i(w_1x_{i1}+w_2x_{i2})))+\frac{1}{2}C||w||^2 \rightarrow min_{w_1,w_2}$

Здесь xi1 и xi2 — значение первого и второго признаков соответственно на объекте xi. В этом задании мы будем рассматривать алгоритмы без свободного члена, чтобы упростить работу.

Градиентный шаг для весов будет заключаться в одновременном обновлении весов w1 и w2 по следующим формулам (проверьте сами, что здесь действительно выписана производная нашего функционала):

$w_1:=w_1+k\frac{1}{l}\sum^{l}_{i=1}y_ix_{i1}\left( 1-\frac{1}{1+exp(-y_1(w_1x_i1+w_2x_{i2}))}\right)-kCw_1$

$w_2:=w_2+k\frac{1}{l}\sum^{l}_{i=1}y_ix_{i1}\left( 1-\frac{1}{1+exp(-y_1(w_1x_i1+w_2x_{i2}))}\right)-kCw_2$


Здесь k — размер шага.

Линейные методы могут переобучаться и давать плохое качество из-за различных проблем в данных: мультиколлинеарности, зашумленности и т.д. Чтобы избежать этого, следует использовать регуляризацию — она позволяет понизить сложность модели и не допустить переобучения. Сила регуляризации определяется коэффициентом C в формулах, указанных выше.

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('data-logistic.csv', header=None)

In [3]:
y = data[0]
X = data[[1,2]]

In [4]:
w = np.array([0,0])
k=0.1
l = len(X)


In [5]:
w = np.array([0,1])

In [6]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def gradient_descent(X,y,w,k,l,C):
    #gradient descent
    w = w + k*1/l*np.sum(np.dot(np.dot(X.T,y),(1-sigmoid(np.dot(X,w))).T),axis=1)-k*C*w.T
    return w

In [7]:
k=0.1
l = len(X)
C = 10

In [33]:
w = np.array([0,0]).reshape(2,1)
y = np.array(y).reshape(205,1)
X = np.array(X)
#sanity check
print(f'X shape is {X.shape}')
print(f'y shape is {y.shape}')
print(f'w shape is {w.shape}')

X shape is (205, 2)
y shape is (205, 1)
w shape is (2, 1)


In [45]:
#gradient descent
w = w + (k*1/l*np.sum(np.dot(np.dot(X.T,y),(1-sigmoid(np.dot(X,w))).T),axis=1)-k*C*w.T).T

In [43]:
a = np.sum(np.dot(np.dot(X.T,y),(1-sigmoid(np.dot(X,w))).T),axis=1)/l*k - k*C*w.T

In [46]:
w.shape

(2, 1)

In [12]:
# решение взято с интернета

In [52]:
import pandas as pd
import numpy as np
import sklearn.metrics
import math

rawData = pd.read_csv('data-logistic.csv', header=None)
X_ = rawData.values[:,1:]
y_ = rawData.values[:,:1].T[0]

w = np.array([0,0]).reshape(2,1)
y_ = np.array(y).reshape(205,1)
X_ = np.array(X)


def sigmoid (x):
    return 1.0/1+np.exp(-x)

def distance(a,b):
    return np.sqrt(np.square(a[0]-b[0])+np.square(a[1]-b[1]))

def log_regression(X,y,k,w,C, epsilon, max_iter):
    
    w1,w2 = w
    
    for i in range(max_iter):
        w1new = w1 + k*np.mean(y*X[:,0]*(1-(1./(1+np.exp(-y*(w1*X[:,0]+w2*X[:,1]))))))-k*C*w1
        w2new = w2 + k*np.mean(y*X[:,0]*(1-(1./(1+np.exp(-y*(w1*X[:,0]+w2*X[:,1]))))))-k*C*w2
        if distance((w1new,w2new),(w1,w2)) < epsilon:
            break
        w1,w2 = w1new,w2new
        
    predictions = []
    for i in range(len(X)):
        t1 = -w1*X[i,0] - w2*X[i,1]
        s = sigmoid(t1)
        predictions.append(s)
    return predictions


p0 = log_regression(X_,y_, 0.1, w, 0, 0.00001, 10000)
p1 = log_regression(X_,y_, 0.1, w, 10, 0.00001, 10000)

print(sklearn.metrics.roc_auc_score(y,p0))
print(sklearn.metrics.roc_auc_score(y,p1))

0.9355238095238094
0.9355238095238094


(205, 2)